### Comments

**HAS BEEN DONE**
* Jakart vs Cosine vs Euclidian for features
* Logistic regression simple et tuné + filtres --> ça marche assez bien!!
* try different architecture -> testé combinaison de linear et GCNN / APPNP => GCNN avec une linear avant et deux linear apres = best


**DONE 28.12.2019**

* Trasformer some features because skewed --> OK, seems better
* Use genre as feature as well --> OK are we sure we can do it? 
* Grid search for the Laplacian Polynomial: learning rate, p_order, dropout, weight_decay --> maximiser en fonction du F1 sur le val OK 
* Grid search for the Linear_GNN: learning rate, first_layer_size, hidden_size, weight_decay --> maximiser en fonction du F1 sur le val set  OK
* See/tune the impact of weight decay --> OK (added in the Grid search)



**TO DO**
* Finer Grid Search?
* Does it make sense to tune the first_layer_size? And the middle one?
* Skip Connections ? 
* 2 Grid Search again to recheck for standardization effect
* Try to compute a Random classifier, do it 20 times and see what is the F1 score in this case, or is the baseline we have enough
* Explore/tune different NN architectures: e.g. Pure_GNN and Simple_APPNP --> to compare with the tuned Linear_GNN



**TO DO** --> Ant
* add building of model with best parameters after grid search
* Then add testing
* do LP and GNN in grid search for both Standardize and Non-standardize data
* 20 prediction of a random classifier to check F1 score at random

# Machine Learning

In [59]:
import numpy as np
import pandas as pd
import pickle

import sklearn.metrics
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_fscore_support, accuracy_score
from sklearn.utils import resample
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import random

import dgl.function as fn
from dgl import DGLGraph
import dgl.nn.pytorch as dgl_nn
import dgl.transform as dgl_transform

import time

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

Data_path = 'Data/'

### Loading the similarity matrix and generate graph

In [2]:
# Load similarity matrix
file = open(Data_path + 'Adjacency_matrix_all025.pickle', 'rb')
adj_mat =  pickle.load(file)

# Generate graph
G = DGLGraph(graph_data=adj_mat)
G = dgl_transform.add_self_loop(G) # we are sure of doing this?

### Loading features and labels

In [3]:
# Features
features_df = pd.read_csv(Data_path + 'features.csv',index_col=0).drop(columns = ['title'])

# To consider also genres in the features comment this:
#features_df = features_df[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']]


# Labels
labels_df = pd.read_csv(Data_path + 'labels.csv',index_col=0).drop(columns = ['title', ])
IMDB_nom = labels_df['Nominations'].copy()
IMDB_nom.loc[IMDB_nom > 0] = 1
# Checking class imbalance
IMDB_nom.value_counts() # 18.263 % of CLASS 1

0.0    3728
1.0    1074
Name: Nominations, dtype: int64

In [4]:
# If you don't want to transform the features don't run the following section and uncomment this:

#transformed_features = features_df.copy()

### Features engineering

In [4]:
def feat_transform(x):
    trasformed_x = np.zeros(len(x))
    for i in range(len(x)):
        if x[i]>0:
            trasformed_x[i] = np.log(x[i]) # could try also sqrt
        else:
            trasformed_x[i] = x[i]
    return trasformed_x

In [5]:
# Only budget, popularity, revenue and vote_count are very skewed
features_to_transform = features_df[['budget', 'popularity', 'revenue', 'vote_count']]
feat_names = ['budget', 'popularity', 'revenue', 'vote_count']
transformed_feat = features_df.copy()
for name in feat_names:
    transformed_feat[name] = feat_transform(features_to_transform[name])
    

### Generate masks and split train, val, test

In [6]:
sss1 = StratifiedShuffleSplit(n_splits=1, train_size=0.8) #random_state=0
sss2 = StratifiedShuffleSplit(n_splits=1, train_size=0.8) #random_state=0

for prov_index, test_index in sss1.split(transformed_feat, IMDB_nom.values):
    prov_mask = prov_index
    test_mask = test_index

for train_index, val_index in sss2.split(transformed_feat.iloc[prov_mask], IMDB_nom.values[prov_mask]):
    train_mask = train_index
    val_mask = val_index

### Create tensors 

In [7]:
# features
tensor_data = torch.FloatTensor(transformed_feat.values)

# labels
tensor_labels = torch.LongTensor(IMDB_nom.values)

### Standardize 

In [9]:
# Standardizing the data increases performances (73 -> 82), in terms of repartition bit more imbalanced
# but still increase of performance for class 1 samples

# To recheck this in terms of F1 score!!!!

#scaler = StandardScaler()
#transformed_feat.iloc[train_mask] = scaler.fit_transform(transformed_feat.iloc[train_mask].to_numpy())
#transformed_feat.iloc[val_mask] = scaler.transform(transformed_feat.iloc[val_mask].to_numpy())
#transformed_feat.iloc[test_mask] = scaler.transform(transformed_feat.iloc[test_mask].to_numpy())

## Random Classifier 
as baseline

In [69]:
def random_classifier(X):
    """
    Binary classification at random
    INPUT
        |---- X (2D numpy array) the input data (the sample dimension is the first one)
    OUTPUT 
        |---- y (1D numpy array) the random prediction for each sample
    """
    return np.random.randint(0, 2, X.shape[0])

In [111]:
def random_classification(data, labels, mask, name, N):
    precision, recall, f1score, accuracy, CM = [], [], [], [], []
    for _ in range(N):
        rand_pred = random_classifier(data[mask,:])
        pre,rec,f1,sup = precision_recall_fscore_support(labels[mask], rand_pred)
        acc = accuracy_score(labels[mask], rand_pred)
        precision.append(pre[1])
        recall.append(rec[1])
        f1score.append(f1[1])
        accuracy.append(acc)
        CM.append(confusion_matrix(labels[mask], rand_pred))

    print(f'Random Classifier on the {name} set \n'+'-'*80)
    print(f'\t |---- Precision {np.mean(precision):.2%} +/- {np.std(precision):.2%} ')
    print(f'\t |---- Recall {np.mean(recall):.2%} +/- {np.std(recall):.2%}')
    print(f'\t |---- F1-score {np.mean(f1score):.2%} +/- {np.std(f1score):.2%}')
    print(f'\t |---- Accuracy {np.mean(accuracy):.2%} +/- {np.std(accuracy):.2%}')
    print(f'\t \n{np.mean(CM, axis=0)} \n')

In [118]:
N = 50
random_classification(tensor_data, tensor_labels, train_mask, 'Train', N)
random_classification(tensor_data, tensor_labels, val_mask, 'Validation', N)
random_classification(tensor_data, tensor_labels, test_mask, 'Test', N)

Random Classifier on the Train set 
--------------------------------------------------------------------------------
	 |---- Precision 24.23% +/- 0.82% 
	 |---- Recall 49.86% +/- 2.10%
	 |---- F1-score 32.61% +/- 1.17%
	 |---- Accuracy 49.98% +/- 0.79%
	 
[[1163.4  1162.6 ]
 [ 374.04  371.96]] 

Random Classifier on the Validation set 
--------------------------------------------------------------------------------
	 |---- Precision 25.87% +/- 1.19% 
	 |---- Recall 50.09% +/- 2.83%
	 |---- F1-score 34.11% +/- 1.61%
	 |---- Accuracy 49.95% +/- 1.49%
	 
[[284.42 285.58]
 [ 99.32  99.68]] 

Random Classifier on the Test set 
--------------------------------------------------------------------------------
	 |---- Precision 22.47% +/- 1.36% 
	 |---- Recall 50.11% +/- 3.46%
	 |---- F1-score 31.02% +/- 1.91%
	 |---- Accuracy 50.15% +/- 1.58%
	 
[[374.2  371.8 ]
 [107.26 107.74]] 



### Building Logistic Regression

In [10]:
clf = LogisticRegression(C= 1,random_state = 0,solver = 'lbfgs').fit(transformed_feat.iloc[train_mask].to_numpy(),IMDB_nom.values[train_mask])
train_pred = clf.predict(transformed_feat.iloc[train_mask].to_numpy())
test_pred = clf.predict(transformed_feat.iloc[test_mask].to_numpy())

tr_pre,tr_rec,tr_f1,tr_sup = precision_recall_fscore_support(train_pred,IMDB_nom.values[train_mask])
print('Training set:')
print('>>> Precision: {:0.4}'.format(tr_pre[1]))
print('>>> Recall: {:0.4}'.format(tr_rec[1]))
print('>>> F1: {:0.4}'.format(tr_f1[1]))
print('>>> Support: {:}'.format(tr_sup[1]))
print('')

test_pre,test_rec,test_f1,test_sup = precision_recall_fscore_support(test_pred,IMDB_nom.values[test_mask])
print('Test set:')
print('>>> Precision: {:0.4}'.format(test_pre[1]))
print('>>> Recall: {:0.4}'.format(test_rec[1]))
print('>>> F1: {:0.4}'.format(test_f1[1]))
print('>>> Support: {:}'.format(test_sup[1]))

Training set:
>>> Precision: 0.3995
>>> Recall: 0.6932
>>> F1: 0.5068
>>> Support: 427

Test set:
>>> Precision: 0.3907
>>> Recall: 0.6829
>>> F1: 0.497
>>> Support: 123


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [11]:
# Confusion Matrix on validation classification
#disp = sklearn.metrics.plot_confusion_matrix(clf, transformed_feat.iloc[val_mask],IMDB_nom.values[val_mask],cmap=plt.cm.Blues,display_labels = ['Not Nominated','Nominated'],normalize='true')


### Building Logistic Regression & Graph Filtering

In [12]:
# Plot confusion matrix when classifier not available only confusion matrix
def confusion_matrix(matrix):
    figure = plt.figure()
    axes = figure.add_subplot(111)
    test = axes.matshow(matrix, cmap = plt.cm.get_cmap('Blues'));
    axes.set_yticklabels(['','Not Nominated','Nominated'],style='italic')
    axes.set_xticklabels(['','Not Nominated','Nominated'],style='italic')
    axes.set_ylabel('True Label')
    axes.set_xlabel('Predicted Label')
    figure.colorbar(test)
    for (j,i),label in np.ndenumerate(matrix):
        axes.text(i,j,np.round(label,3),ha='center',va='center',color = 'grey')
    plt.show()

In [13]:
class LaplacianPolynomial(nn.Module):
    def __init__(self,
                 in_feats: int,
                 out_feats: int,
                 k: int,
                 dropout_prob: float,
                 norm=True):
        super().__init__()
        self._in_feats = in_feats
        self._out_feats = out_feats
        self._k = k
        self._norm = norm
        # Contains the weights learned by the Laplacian polynomial
        self.pol_weights = nn.Parameter(torch.Tensor(self._k + 1))
        # Contains the weights learned by the logistic regression (without bias)
        self.logr_weights = nn.Parameter(torch.Tensor(in_feats, out_feats))
        self.dropout = nn.Dropout(p=dropout_prob)
        self.reset_parameters()

    def reset_parameters(self):
        """Reinitialize learnable parameters."""
        torch.manual_seed(0)
        torch.nn.init.xavier_uniform_(self.logr_weights, gain=0.01)
        torch.nn.init.normal_(self.pol_weights, mean=0.0, std=1e-3)

    def forward(self, graph, feat):
        r"""Compute graph convolution.

        Notes
        -----
        * Input shape: :math:`(N, *, \text{in_feats})` where * means any number of additional
          dimensions, :math:`N` is the number of nodes.
        * Output shape: :math:`(N, *, \text{out_feats})` where all but the last dimension are
          the same shape as the input.

        Parameters
        ----------
        graph (DGLGraph) : The graph.
        feat (torch.Tensor): The input feature

        Returns
        -------
        (torch.Tensor) The output feature
        """
        feat = self.dropout(feat)
        graph = graph.local_var()
        
        # D^(-1/2)
        norm = torch.pow(graph.in_degrees().float().clamp(min=1), -0.5)
        shp = norm.shape + (1,) * (feat.dim() - 1)
        norm = torch.reshape(norm, shp)

        # mult W first to reduce the feature size for aggregation.
        feat = torch.matmul(feat, self.logr_weights) # X*Teta

        result = self.pol_weights[0] * feat.clone() # a0*L^0*X*Teta <-- fisrt polynomial weight a0 * L^0 * x

        for i in range(1, self._k + 1): # get the next polynomial coefficient (a1*L^1, a2*L^2, ..... ak*L^k) 
            old_feat = feat.clone()
            if self._norm:
                feat = feat * norm
            graph.ndata['h'] = feat
            # Feat is not modified in place
            graph.update_all(fn.copy_src(src='h', out='m'),
                             fn.sum(msg='m', out='h')) # update all nodes with msg function copy_src (get data from source node) and reduce function sum
            if self._norm:
                graph.ndata['h'] = graph.ndata['h'] * norm

            feat = old_feat - graph.ndata['h']
            result += self.pol_weights[i] * feat

        return result

    def extra_repr(self):
        """Set the extra representation of the module,
        which will come into effect when printing the model.
        """
        summary = 'in={_in_feats}, out={_out_feats}'
        summary += ', normalization={_norm}'
        return summary.format(**self.__dict__)

In [14]:
def train(model, g, features, labels, train_mask, loss_fcn, optimizer):
    """ 
    DESCRIPTION : Train and update model classification performances with training set
    INPUT:
        |--- model: [] classification model to train
        |--- g: [DGLgraph] DeepGraphLearning graph object
        |--- features: [FloatTensor] 2D tensor containing samples' features
        |--- labels: [LongTensor] 1D tensor containing samples' labels (0-1)
        |--- train_mask: [np.array] indices of training set
        |--- loss_fcn: pytorch loss function chosen for model training
        |--- optimizer: pytorch model optimizer 
    OUTPUT:
        |--- loss: [float] value of loss function for the model at current state
    """
    model.train()  
    
    pred = model(g, features)[train_mask] # prediction
    loss = loss_fcn(pred, labels[train_mask])
    optimizer.zero_grad()    
    loss.backward()

    optimizer.step()
    #_, indices = torch.max(pred, dim=1)
    #correct = torch.sum(indices == labels[train_mask])
    #acc = correct.item() * 1.0 / len(labels[train_mask]) #not the best metric
    
    #C = sklearn.metrics.confusion_matrix(tensor_labels[train_mask], indices.numpy(), labels=[0,1], sample_weight=None, normalize='true')

    #return loss, acc, C
    return loss
    
def evaluate(model, g, features, mask, labels):
    """ 
    DESCRIPTION : Evaluate model classification performance on validation set 
    INPUT:
        |--- model: [] classification model to evaluate
        |--- g: [DGLgraph] DeepGraphLearning graph object
        |--- features: [FloatTensor] 2D tensor containing samples' features
        |--- labels: [LongTensor] 1D tensor containing samples' labels (0-1)
        |--- mask: [np.array] indices of validation set
    OUTPUT:
        |--- acc: [float] classification accuracy
        |--- recall: [float] classification recall
        |--- precision: [float] classification precision
        |--- f1: [float] classification f1 score
    """
    model.eval() 
    
    with torch.no_grad():
        pred = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(pred, dim=1)
        
        pre,rec,f1,sup = precision_recall_fscore_support(labels,indices.numpy())
        #correct = torch.sum(indices == labels)
        #acc = correct.item() * 1.0 / len(labels) #not the best metric
        #f1 = f1_score(labels, indices)
        #recall = recall_score(labels, indices)
        #precision = precision_score(labels, indices)
        
        C = sklearn.metrics.confusion_matrix(labels, indices.numpy())
        
        return pre[1], rec[1], f1[1], sup[1], C

##### Results of optimisation : 

Optimisation performed by looking at validation accuracy and distribution of errors across classes using confusion matrix
- polynomial order : increase of the order tends to increase the instability of performances accross epochs, no strong impact on filter final shape -> tradeoff complexity/stability at 3
- learning rate: small shift towards very unbalanced error, higher learning rate enabled to get a better trade-off between accuracy and distribution of error -> 0.2
- number of epochs : event of strong instabilities across trials whatever parameters; strong instabilities allows better balance of errors but weaker accuracies; around 1500 period of stable learning
- dropout : increase generates instabilities, tradeoff between accuracy and distribution of errors at 0.
=> Final filter is basically always the same sort of shape as shown below

### Tuning of Laplacian Polynomial

In [15]:
n_epochs = 1000

# To tune
learning_rate = [3e-1,5e-1,5e-2,1e-2]
pol_order = [2,3,4] 
p_dropout = [0.2,0.3,0.4] 
weight_decay = [0,5e-5,5e-6] # by default = 0


n_classes = 2
in_feats=tensor_data.shape[1]

true_ratio = 1074/4802 # <-- fraction of Nominations
weights_loss = torch.FloatTensor([true_ratio, 1-true_ratio]) # to rebalance

# results with features transformed and  without standardization : 
# best_F1 = 0.5662, learning_rate = 0.3, pol_order = 3, p_dropout = 0.2, weight_decay = 5e-5

In [16]:
def grid_search_LP(learning_rate, pol_order,p_dropout,weight_decay):
    

    performances = torch.zeros(len(learning_rate),len(pol_order),len(p_dropout),len(weight_decay))
    for l, lr_ in enumerate(learning_rate):
        for p, p_order in enumerate(pol_order):
            for d, dropout in enumerate(p_dropout):
                for w, weight in enumerate(weight_decay):
                        
                    model = LaplacianPolynomial(in_feats, n_classes, p_order, dropout)

                    loss_fcn = torch.nn.CrossEntropyLoss(weight=weights_loss)
                    optimizer = torch.optim.Adam(model.parameters(),lr=lr_, weight_decay = weight)
                    losses_tr = []
                        
                    for epoch in range(n_epochs):
                        loss = train(model, G, tensor_data, tensor_labels, train_mask, loss_fcn, optimizer)
                        losses_tr.append(loss.item())
                        pre, rec, f1, sup, C = evaluate(model, G, tensor_data, val_mask, tensor_labels) 
                        performances[l,p,d,w] = f1
                        if (epoch+1)%100 == 0:
                            print("Epoch {:05d} | Train Loss {:.4f} | Val precision {:.4%} | Val recall {:.4%} | Val F1 {:.4%}". format(epoch+1, loss.item(), pre, rec, f1))
                            

        best_performance = torch.max(performances)
        best_idx = (performances == best_performance).nonzero();
            
        best_lr = learning_rate[best_idx[0,0]]
        best_p_order = pol_order[best_idx[0,1]]
        best_dropout = p_dropout[best_idx[0,2]]
        best_weight = weight_decay[best_idx[0,3]]
                
        results = [best_performance, best_lr,  best_p_order, best_dropout,best_weight]
    return results

In [17]:
results_LP = grid_search_LP(learning_rate, pol_order,p_dropout,weight_decay)
print(results_LP)

# results with features transformed and  without standardization : 
# best_F1 = 0.5662, learning_rate = 0.3, pol_order = 3, p_dropout = 0.2, weight_decay = 5e-5

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 00100 | Train Loss 0.6797 | Val precision 31.3894% | Val recall 89.7059% | Val F1 46.5057%
Epoch 00200 | Train Loss 0.6467 | Val precision 32.5424% | Val recall 94.1176% | Val F1 48.3627%
Epoch 00300 | Train Loss 0.6088 | Val precision 42.4403% | Val recall 78.4314% | Val F1 55.0775%
Epoch 00400 | Train Loss 0.6006 | Val precision 44.5428% | Val recall 74.0196% | Val F1 55.6169%
Epoch 00500 | Train Loss 0.5977 | Val precision 44.6483% | Val recall 71.5686% | Val F1 54.9906%
Epoch 00600 | Train Loss 0.6015 | Val precision 42.4403% | Val recall 78.4314% | Val F1 55.0775%
Epoch 00700 | Train Loss 0.5980 | Val precision 45.4829% | Val recall 71.5686% | Val F1 55.6190%
Epoch 00800 | Train Loss 0.5968 | Val precision 44.3750% | Val recall 69.6078% | Val F1 54.1985%
Epoch 00900 | Train Loss 0.5983 | Val precision 44.5455% | Val recall 72.0588% | Val F1 55.0562%
Epoch 01000 | Train Loss 0.5988 | Val precision 45.4545% | Val recall 68.6275% | Val F1 54.6875%
Epoch 00100 | Train Loss 0.626

Epoch 00600 | Train Loss 0.6163 | Val precision 43.7063% | Val recall 61.2745% | Val F1 51.0204%
Epoch 00700 | Train Loss 0.6148 | Val precision 42.9775% | Val recall 75.0000% | Val F1 54.6429%
Epoch 00800 | Train Loss 0.6066 | Val precision 42.4501% | Val recall 73.0392% | Val F1 53.6937%
Epoch 00900 | Train Loss 0.6145 | Val precision 44.1696% | Val recall 61.2745% | Val F1 51.3347%
Epoch 01000 | Train Loss 0.6124 | Val precision 42.7536% | Val recall 57.8431% | Val F1 49.1667%
Epoch 00100 | Train Loss 0.6195 | Val precision 41.2256% | Val recall 72.5490% | Val F1 52.5755%
Epoch 00200 | Train Loss 0.5986 | Val precision 45.0450% | Val recall 73.5294% | Val F1 55.8659%
Epoch 00300 | Train Loss 0.6053 | Val precision 44.7531% | Val recall 71.0784% | Val F1 54.9242%
Epoch 00400 | Train Loss 0.6007 | Val precision 43.8202% | Val recall 76.4706% | Val F1 55.7143%
Epoch 00500 | Train Loss 0.5978 | Val precision 45.3731% | Val recall 74.5098% | Val F1 56.4007%
Epoch 00600 | Train Loss 0.600

Epoch 00100 | Train Loss 0.6738 | Val precision 28.5112% | Val recall 99.5098% | Val F1 44.3231%
Epoch 00200 | Train Loss 0.6118 | Val precision 43.4659% | Val recall 75.0000% | Val F1 55.0360%
Epoch 00300 | Train Loss 0.6144 | Val precision 43.2177% | Val recall 67.1569% | Val F1 52.5912%
Epoch 00400 | Train Loss 0.6089 | Val precision 42.3077% | Val recall 70.0980% | Val F1 52.7675%
Epoch 00500 | Train Loss 0.6090 | Val precision 42.3881% | Val recall 69.6078% | Val F1 52.6902%
Epoch 00600 | Train Loss 0.6129 | Val precision 43.3140% | Val recall 73.0392% | Val F1 54.3796%
Epoch 00700 | Train Loss 0.6099 | Val precision 43.5811% | Val recall 63.2353% | Val F1 51.6000%
Epoch 00800 | Train Loss 0.6064 | Val precision 42.2572% | Val recall 78.9216% | Val F1 55.0427%
Epoch 00900 | Train Loss 0.6137 | Val precision 45.3020% | Val recall 66.1765% | Val F1 53.7849%
Epoch 01000 | Train Loss 0.6089 | Val precision 43.8127% | Val recall 64.2157% | Val F1 52.0875%
Epoch 00100 | Train Loss 0.613

Epoch 00600 | Train Loss 0.6110 | Val precision 42.8977% | Val recall 74.0196% | Val F1 54.3165%
Epoch 00700 | Train Loss 0.6292 | Val precision 43.3594% | Val recall 54.4118% | Val F1 48.2609%
Epoch 00800 | Train Loss 0.6196 | Val precision 37.6906% | Val recall 84.8039% | Val F1 52.1870%
Epoch 00900 | Train Loss 0.6116 | Val precision 43.0233% | Val recall 54.4118% | Val F1 48.0519%
Epoch 01000 | Train Loss 0.6111 | Val precision 39.8104% | Val recall 82.3529% | Val F1 53.6741%
Epoch 00100 | Train Loss 0.6376 | Val precision 35.1190% | Val recall 86.7647% | Val F1 50.0000%
Epoch 00200 | Train Loss 0.6139 | Val precision 44.1640% | Val recall 68.6275% | Val F1 53.7428%
Epoch 00300 | Train Loss 0.6156 | Val precision 42.3592% | Val recall 77.4510% | Val F1 54.7660%
Epoch 00400 | Train Loss 0.6076 | Val precision 42.2857% | Val recall 72.5490% | Val F1 53.4296%
Epoch 00500 | Train Loss 0.6190 | Val precision 44.0252% | Val recall 68.6275% | Val F1 53.6398%
Epoch 00600 | Train Loss 0.608

...

Epoch 00100 | Train Loss 0.6726 | Val precision 30.6947% | Val recall 93.1373% | Val F1 46.1725%
Epoch 00200 | Train Loss 0.6073 | Val precision 43.5897% | Val recall 75.0000% | Val F1 55.1351%
Epoch 00300 | Train Loss 0.6109 | Val precision 42.9012% | Val recall 68.1373% | Val F1 52.6515%
Epoch 00400 | Train Loss 0.6075 | Val precision 42.3592% | Val recall 77.4510% | Val F1 54.7660%
Epoch 00500 | Train Loss 0.6040 | Val precision 42.4084% | Val recall 79.4118% | Val F1 55.2901%
Epoch 00600 | Train Loss 0.6053 | Val precision 44.0895% | Val recall 67.6471% | Val F1 53.3849%
Epoch 00700 | Train Loss 0.6009 | Val precision 42.9730% | Val recall 77.9412% | Val F1 55.4007%
Epoch 00800 | Train Loss 0.6006 | Val precision 45.5696% | Val recall 70.5882% | Val F1 55.3846%
Epoch 00900 | Train Loss 0.6061 | Val precision 46.0432% | Val recall 62.7451% | Val F1 53.1120%
Epoch 01000 | Train Loss 0.6064 | Val precision 44.7761% | Val recall 73.5294% | Val F1 55.6586%
Epoch 00100 | Train Loss 0.680

Epoch 00600 | Train Loss 0.6067 | Val precision 44.5161% | Val recall 67.6471% | Val F1 53.6965%
Epoch 00700 | Train Loss 0.6057 | Val precision 38.3073% | Val recall 84.3137% | Val F1 52.6799%
Epoch 00800 | Train Loss 0.6039 | Val precision 42.3377% | Val recall 79.9020% | Val F1 55.3480%
Epoch 00900 | Train Loss 0.6082 | Val precision 44.1176% | Val recall 73.5294% | Val F1 55.1471%
Epoch 01000 | Train Loss 0.6055 | Val precision 44.3787% | Val recall 73.5294% | Val F1 55.3506%
Epoch 00100 | Train Loss 0.6102 | Val precision 42.7778% | Val recall 75.4902% | Val F1 54.6099%
Epoch 00200 | Train Loss 0.6083 | Val precision 44.6809% | Val recall 72.0588% | Val F1 55.1595%
Epoch 00300 | Train Loss 0.6115 | Val precision 44.4776% | Val recall 73.0392% | Val F1 55.2876%
Epoch 00400 | Train Loss 0.6035 | Val precision 43.6416% | Val recall 74.0196% | Val F1 54.9091%
Epoch 00500 | Train Loss 0.6101 | Val precision 44.1696% | Val recall 61.2745% | Val F1 51.3347%
Epoch 00600 | Train Loss 0.604

Epoch 00100 | Train Loss 0.6233 | Val precision 43.9759% | Val recall 71.5686% | Val F1 54.4776%
Epoch 00200 | Train Loss 0.6122 | Val precision 44.6483% | Val recall 71.5686% | Val F1 54.9906%
Epoch 00300 | Train Loss 0.6107 | Val precision 45.3416% | Val recall 71.5686% | Val F1 55.5133%
Epoch 00400 | Train Loss 0.6078 | Val precision 45.2012% | Val recall 71.5686% | Val F1 55.4080%
Epoch 00500 | Train Loss 0.6018 | Val precision 44.7368% | Val recall 66.6667% | Val F1 53.5433%
Epoch 00600 | Train Loss 0.6058 | Val precision 43.6578% | Val recall 72.5490% | Val F1 54.5120%
Epoch 00700 | Train Loss 0.6010 | Val precision 44.0299% | Val recall 57.8431% | Val F1 50.0000%
Epoch 00800 | Train Loss 0.6008 | Val precision 42.2460% | Val recall 77.4510% | Val F1 54.6713%
Epoch 00900 | Train Loss 0.6061 | Val precision 45.2381% | Val recall 65.1961% | Val F1 53.4137%
Epoch 01000 | Train Loss 0.6043 | Val precision 44.5902% | Val recall 66.6667% | Val F1 53.4381%
Epoch 00100 | Train Loss 0.623

Epoch 00600 | Train Loss 0.6048 | Val precision 43.9528% | Val recall 73.0392% | Val F1 54.8803%
Epoch 00700 | Train Loss 0.6009 | Val precision 44.2029% | Val recall 59.8039% | Val F1 50.8333%
Epoch 00800 | Train Loss 0.5998 | Val precision 42.1189% | Val recall 79.9020% | Val F1 55.1607%
Epoch 00900 | Train Loss 0.6068 | Val precision 43.1755% | Val recall 75.9804% | Val F1 55.0622%
Epoch 01000 | Train Loss 0.6050 | Val precision 42.5926% | Val recall 78.9216% | Val F1 55.3265%
Epoch 00100 | Train Loss 0.6274 | Val precision 43.0168% | Val recall 75.4902% | Val F1 54.8043%
Epoch 00200 | Train Loss 0.6065 | Val precision 44.9231% | Val recall 71.5686% | Val F1 55.1985%
Epoch 00300 | Train Loss 0.6087 | Val precision 44.5183% | Val recall 65.6863% | Val F1 53.0693%
Epoch 00400 | Train Loss 0.6068 | Val precision 45.0704% | Val recall 62.7451% | Val F1 52.4590%
Epoch 00500 | Train Loss 0.6032 | Val precision 43.8486% | Val recall 68.1373% | Val F1 53.3589%
Epoch 00600 | Train Loss 0.605

Epoch 00100 | Train Loss 0.6179 | Val precision 44.8598% | Val recall 70.5882% | Val F1 54.8571%
Epoch 00200 | Train Loss 0.6090 | Val precision 43.8596% | Val recall 73.5294% | Val F1 54.9451%
Epoch 00300 | Train Loss 0.6112 | Val precision 42.1875% | Val recall 79.4118% | Val F1 55.1020%
Epoch 00400 | Train Loss 0.6034 | Val precision 44.4805% | Val recall 67.1569% | Val F1 53.5156%
Epoch 00500 | Train Loss 0.6108 | Val precision 44.1270% | Val recall 68.1373% | Val F1 53.5645%
Epoch 00600 | Train Loss 0.6050 | Val precision 43.5530% | Val recall 74.5098% | Val F1 54.9729%
Epoch 00700 | Train Loss 0.6041 | Val precision 42.3497% | Val recall 75.9804% | Val F1 54.3860%
Epoch 00800 | Train Loss 0.6033 | Val precision 44.5428% | Val recall 74.0196% | Val F1 55.6169%
Epoch 00900 | Train Loss 0.6062 | Val precision 43.5294% | Val recall 72.5490% | Val F1 54.4118%
Epoch 01000 | Train Loss 0.6039 | Val precision 43.5185% | Val recall 69.1176% | Val F1 53.4091%
Epoch 00100 | Train Loss 0.618

In [18]:
def polynomial_graph_filter_response(coeff: np.array, lam: np.ndarray):
    """ 
        DESCRIPTION : Compute response of filtering using a polynomial filter 
        INPUT:
            |--- coeff: [np.array] coeffiicients of polynomial filter
            |--- lam: [np.ndarray] eigenvalues 
        OUTPUT:
            |--- response: [np.ndarray] response[i] is the spectral response at frequency lam[i]
    """
    V = np.vander(lam,coeff.shape[0],increasing=True)
    response = V@coeff
    return response

In [19]:
def spectral_decomposition(laplacian: np.ndarray):
    """ 
        DESCRIPTION : Compute spectral decomposition of a graph using the graph Laplacian
        INPUT:
            |--- laplacian: [np.ndarray] graph laplacian 
        OUTPUT:
            |--- lamb: [np.ndarray] containing graph eigenvalues
            |--- U: [np.ndarray] containing corresponding graph eigenvectors
    """
    # compute the eigenvalues and eigenvectors
    if np.allclose(laplacian, laplacian.T, 1e-12):
        lamb, U = np.linalg.eigh(laplacian)
    else:
        lamb, U = np.linalg.eig(laplacian)
        #sort them
        idx = np.argsort(lamb, axis=0)
        lamb = lamb[idx]
        U = U[:,idx]
    
    return lamb, U

In [20]:
def compute_laplacian(adjacency: np.ndarray, normalize: bool):
    """ 
        DESCRIPTION : Compute spectral decomposition of a graph using the graph Laplacian
        INPUT:
            |--- adjacency: [np.ndarray] adjacency matrix of the graph
            |--- normalize: [bool] if normalize laplacian or not
        OUTPUT:
            |--- L: [n x n ndarray] combinatorial or symmetric normalized Laplacian. of the graph 
    """
    # degrees
    I = np.identity(adjacency.shape[0])
    degree = np.sum(adjacency, axis=1)
    # Compute laplacian
    D = I.copy()
    np.fill_diagonal(D, degree)
    L = D - adjacency
    # normalized if requested 
    if normalize:
        D12 = np.where(D > 0, np.power(D, -0.5, where=D>0), 0)
        L = D12 @ L @ D12
        
    return L

### Building Graph Neural Network

In [21]:
#  model: Combine GraphConv layers first then two fully connected layers --> seems less stable over epochs
class Linear_GNN(nn.Module):
    def __init__(self, in_feats: int, out_feats: int, first_layer_size: int, hidden_size: int):
        super().__init__()
        self._in_feats = in_feats
        self._out_feats = out_feats
        self._first_layer_size = first_layer_size
        self._hidden_size = hidden_size
        
        # Layers --> as much GraphConv as diameter --> reach everywhere
        layer_size = 128
        self.linear = nn.Linear(self._in_feats, self._first_layer_size)
        self.gcn1 = dgl_nn.conv.GraphConv(self._first_layer_size, layer_size, activation=F.relu)
        self.gcn2 = dgl_nn.conv.GraphConv(layer_size, layer_size, activation=F.relu)
        self.gcn3 = dgl_nn.conv.GraphConv(layer_size, layer_size, activation=F.relu)
        self.gcn4 = dgl_nn.conv.GraphConv(layer_size, layer_size, activation=F.relu)
        self.gcn5 = dgl_nn.conv.GraphConv(layer_size, layer_size, activation=F.relu)
        self.gcn6 = dgl_nn.conv.GraphConv(layer_size, layer_size, activation=F.relu)
        self.gcn7 = dgl_nn.conv.GraphConv(layer_size, layer_size, activation=F.relu)
        self.gcn8 = dgl_nn.conv.GraphConv(layer_size, layer_size, activation=F.relu)
        self.gcn9 = dgl_nn.conv.GraphConv(layer_size, layer_size, activation=F.relu)
        self.gcn10 = dgl_nn.conv.GraphConv(layer_size, layer_size, activation=F.relu)
        self.gcn11 = dgl_nn.conv.GraphConv(layer_size, layer_size, activation=F.relu)
        self.linear1 = nn.Linear(layer_size, self._hidden_size)
        self.linear2 = nn.Linear(self._hidden_size, self._out_feats)
        
    def forward(self, graph, feat):
        h = F.relu(self.linear(feat))
        h = self.gcn1(graph, h)
        h = self.gcn2(graph, h)
        h = self.gcn3(graph, h)
        h = self.gcn4(graph, h)
        h = self.gcn5(graph, h)
        h = self.gcn6(graph, h)
        h = self.gcn7(graph, h)
        h = self.gcn8(graph, h)
        h = self.gcn9(graph, h)
        h = self.gcn10(graph, h)
        h = self.gcn11(graph, h)
        h = self.linear1(h)
        h = F.relu(h)
        h = self.linear2(h)
        h = F.log_softmax(h, dim=1)
        return h 

# Model : Only GraphConv layers --> seems more stable
class Pure_GNN(nn.Module):
    def __init__(self, in_feats: int, out_feats: int, hidden_size: int):
        super().__init__()
        self._in_feats = in_feats
        self._out_feats = out_feats
        self._hidden_size = hidden_size
        
        # Layers
        self.gcn1 = dgl_nn.conv.GraphConv(self._in_feats, 32, activation=F.relu)
        self.gcn2 = dgl_nn.conv.GraphConv(32, 32, activation=F.relu)
        self.gcn3 = dgl_nn.conv.GraphConv(32, 32, activation=F.relu)
        self.gcn4 = dgl_nn.conv.GraphConv(32, 32, activation=F.relu)
        self.gcn5 = dgl_nn.conv.GraphConv(32, 32, activation=F.relu)
        self.gcn6 = dgl_nn.conv.GraphConv(32, 32, activation=F.relu)
        self.gcn7 = dgl_nn.conv.GraphConv(32, 32, activation=F.relu)
        self.gcn8 = dgl_nn.conv.GraphConv(32, 32, activation=F.relu)
        self.gcn9 = dgl_nn.conv.GraphConv(32, 32, activation=F.relu)
        self.gcn10 = dgl_nn.conv.GraphConv(32, 32, activation=F.relu)
        self.gcn11 = dgl_nn.conv.GraphConv(32,  self._out_feats, activation=None)
        
    def forward(self, graph, feat):
        h = self.gcn1(graph, feat)
        h = self.gcn2(graph, h)
        h = self.gcn3(graph, h)
        h = self.gcn4(graph, h)
        h = self.gcn5(graph, h)
        h = self.gcn6(graph, h)
        h = self.gcn7(graph, h)
        h = self.gcn8(graph, h)
        h = self.gcn9(graph, h)
        h = self.gcn10(graph, h)
        h = self.gcn11(graph, h)
        h = F.log_softmax(h, dim=1)
        return h 

# model : Use and APPNP layer with k=7 (the network diameter) followed by 2 fully connected linears. 
class Simple_APPNP(nn.Module):
    def __init__(self, in_feats: int, out_feats: int, hidden_size: int, k: int):
        super().__init__()
        self._k = k
        self._in_feats = in_feats
        self._out_feats = out_feats
        self._hidden_size = hidden_size
        
        # Layers
        self.appnpconv1 = dgl_nn.conv.APPNPConv(self._k, 0.1, 0) #alpha teleport proba = 0.1 (cf paper)
        self.linear1 = nn.Linear(self._hidden_size, self._hidden_size)
        self.linear2 = nn.Linear(self._hidden_size, self._out_feats)
        
    def forward(self, graph, feat):
        h = self.appnpconv1(graph, feat)
        h = self.linear1(h)
        h = F.relu(h)
        h = self.linear2(h)
        h = F.log_softmax(h, dim=1)
        return h 


### Tuning Graph NN 

In [22]:
# Fixed values
in_feats = tensor_data.shape[1]
out_feats = 2

k = 11 # number of hop (how far to look) usually best to use the network diameter (according to paper)

# Not relevant parameters
n_epochs = 1000

# To tune in the grid search
learning_rate = [1e-3,1e-4,1e-5]
first_layer_size = [16,32,64]
hidden_size = [256,512]
weight_decay = [0,5e-5,5e-6] # by default = 0

# results with features transformed and  without standardization : 
#best F1 = 0.5138, learning_rate = 0.0001, first_layer =  16,  hidden_size = 512, weight_decay = 5e-06]


#lr2 = 2e-5
#lr3 = 8e-6


In [23]:
def grid_search_NN(learning_rate, first_layer_size,hidden_size,weight_decay):
    
    performances = torch.zeros(len(learning_rate),len(first_layer_size),len(hidden_size),len(weight_decay))
    for l, lr_ in enumerate(learning_rate):
        for f, f_layer in enumerate(first_layer_size):
            for h, hidden in enumerate(hidden_size):
                for w, weight in enumerate(weight_decay):
                        
                    model = Linear_GNN(in_feats, out_feats, f_layer, hidden)

                    loss_fcn = torch.nn.CrossEntropyLoss(weight=weights_loss)
                    optimizer = torch.optim.Adam(model.parameters(),lr=lr_, weight_decay=weight)
                    losses_tr = []
                        
                    for epoch in range(n_epochs):
                        loss = train(model, G, tensor_data, tensor_labels, train_mask, loss_fcn, optimizer)
                        losses_tr.append(loss.item())
                        pre, rec, f1, sup, C = evaluate(model, G, tensor_data, val_mask, tensor_labels) 
                        performances[l,f,h,w] = f1
                        if (epoch+1)%100 == 0:
                                print("Epoch {:05d} | Train Loss {:.4f} | Val precision {:.4%} | Val recall {:.4%} | Val F1 {:.4%}". format(epoch+1, loss.item(), pre, rec, f1))
                            

        best_performance = torch.max(performances)
        best_idx = (performances == best_performance).nonzero();
            
        best_lr = learning_rate[best_idx[0,0]]
        best_first_layer = first_layer_size[best_idx[0,1]]
        best_hidden_layer = hidden_size[best_idx[0,2]]
        best_weight = weight_decay[best_idx[0,3]]
                
        results = [best_performance, best_lr, best_first_layer, best_hidden_layer,best_weight]
    return results

In [24]:
results_NN = grid_search_NN(learning_rate, first_layer_size,hidden_size,weight_decay)
print(results_NN)


Epoch 00100 | Train Loss 0.6812 | Val precision 49.2754% | Val recall 16.6667% | Val F1 24.9084%
Epoch 00200 | Train Loss 0.6286 | Val precision 33.6283% | Val recall 74.5098% | Val F1 46.3415%
Epoch 00300 | Train Loss 0.6006 | Val precision 38.5638% | Val recall 71.0784% | Val F1 50.0000%
Epoch 00400 | Train Loss 0.6239 | Val precision 48.0916% | Val recall 30.8824% | Val F1 37.6119%
Epoch 00500 | Train Loss 0.5971 | Val precision 40.7080% | Val recall 67.6471% | Val F1 50.8287%
Epoch 00600 | Train Loss 0.5802 | Val precision 39.1421% | Val recall 71.5686% | Val F1 50.6066%
Epoch 00700 | Train Loss 0.6141 | Val precision 35.9914% | Val recall 81.8627% | Val F1 50.0000%
Epoch 00800 | Train Loss 0.5871 | Val precision 39.8352% | Val recall 71.0784% | Val F1 51.0563%
Epoch 00900 | Train Loss 0.5695 | Val precision 39.7695% | Val recall 67.6471% | Val F1 50.0907%
Epoch 01000 | Train Loss 0.5812 | Val precision 38.4040% | Val recall 75.4902% | Val F1 50.9091%
Epoch 00100 | Train Loss 0.652

Epoch 00600 | Train Loss 0.5872 | Val precision 38.3420% | Val recall 72.5490% | Val F1 50.1695%
Epoch 00700 | Train Loss 0.6047 | Val precision 38.7006% | Val recall 67.1569% | Val F1 49.1039%
Epoch 00800 | Train Loss 0.6910 | Val precision 28.9370% | Val recall 72.0588% | Val F1 41.2921%
Epoch 00900 | Train Loss 0.6122 | Val precision 35.1351% | Val recall 70.0980% | Val F1 46.8085%
Epoch 01000 | Train Loss 0.6007 | Val precision 37.1968% | Val recall 67.6471% | Val F1 48.0000%
Epoch 00100 | Train Loss 0.6238 | Val precision 45.6044% | Val recall 40.6863% | Val F1 43.0052%
Epoch 00200 | Train Loss 0.6034 | Val precision 35.7631% | Val recall 76.9608% | Val F1 48.8336%
Epoch 00300 | Train Loss 0.6161 | Val precision 36.9775% | Val recall 56.3725% | Val F1 44.6602%
Epoch 00400 | Train Loss 0.5935 | Val precision 36.7150% | Val recall 74.5098% | Val F1 49.1909%
Epoch 00500 | Train Loss 0.5752 | Val precision 43.3333% | Val recall 63.7255% | Val F1 51.5873%
Epoch 00600 | Train Loss 0.588

Epoch 00100 | Train Loss 0.6973 | Val precision 33.3333% | Val recall 0.9804% | Val F1 1.9048%
Epoch 00200 | Train Loss 0.6502 | Val precision 39.1144% | Val recall 51.9608% | Val F1 44.6316%
Epoch 00300 | Train Loss 0.6082 | Val precision 36.8146% | Val recall 69.1176% | Val F1 48.0409%
Epoch 00400 | Train Loss 0.5969 | Val precision 39.1421% | Val recall 71.5686% | Val F1 50.6066%
Epoch 00500 | Train Loss 0.5884 | Val precision 39.8876% | Val recall 69.6078% | Val F1 50.7143%
Epoch 00600 | Train Loss 0.5891 | Val precision 40.4984% | Val recall 63.7255% | Val F1 49.5238%
Epoch 00700 | Train Loss 0.5737 | Val precision 39.8176% | Val recall 64.2157% | Val F1 49.1557%
Epoch 00800 | Train Loss 0.5735 | Val precision 40.2597% | Val recall 60.7843% | Val F1 48.4375%
Epoch 00900 | Train Loss 0.5843 | Val precision 40.6154% | Val recall 64.7059% | Val F1 49.9055%
Epoch 01000 | Train Loss 0.5687 | Val precision 45.0000% | Val recall 57.3529% | Val F1 50.4310%
Epoch 00100 | Train Loss 0.6568 

Epoch 00600 | Train Loss 0.6086 | Val precision 35.4217% | Val recall 72.0588% | Val F1 47.4960%
Epoch 00700 | Train Loss 0.6029 | Val precision 36.8831% | Val recall 69.6078% | Val F1 48.2173%
Epoch 00800 | Train Loss 0.5991 | Val precision 37.0844% | Val recall 71.0784% | Val F1 48.7395%
Epoch 00900 | Train Loss 0.5976 | Val precision 36.9231% | Val recall 70.5882% | Val F1 48.4848%
Epoch 01000 | Train Loss 0.5968 | Val precision 35.2144% | Val recall 76.4706% | Val F1 48.2226%
Epoch 00100 | Train Loss 0.6616 | Val precision 32.6754% | Val recall 73.0392% | Val F1 45.1515%
Epoch 00200 | Train Loss 0.6239 | Val precision 39.5904% | Val recall 56.8627% | Val F1 46.6801%
Epoch 00300 | Train Loss 0.6216 | Val precision 39.2739% | Val recall 58.3333% | Val F1 46.9428%
Epoch 00400 | Train Loss 0.6103 | Val precision 38.2090% | Val recall 62.7451% | Val F1 47.4954%
Epoch 00500 | Train Loss 0.6076 | Val precision 35.3075% | Val recall 75.9804% | Val F1 48.2115%
Epoch 00600 | Train Loss 0.608

Epoch 00100 | Train Loss 0.6776 | Val precision 36.7003% | Val recall 53.4314% | Val F1 43.5130%
Epoch 00200 | Train Loss 0.6367 | Val precision 33.9806% | Val recall 68.6275% | Val F1 45.4545%
Epoch 00300 | Train Loss 0.6302 | Val precision 35.7309% | Val recall 75.4902% | Val F1 48.5039%
Epoch 00400 | Train Loss 0.6233 | Val precision 36.4611% | Val recall 66.6667% | Val F1 47.1404%
Epoch 00500 | Train Loss 0.6188 | Val precision 36.1364% | Val recall 77.9412% | Val F1 49.3789%
Epoch 00600 | Train Loss 0.6093 | Val precision 37.8082% | Val recall 67.6471% | Val F1 48.5062%
Epoch 00700 | Train Loss 0.6045 | Val precision 37.9032% | Val recall 69.1176% | Val F1 48.9583%
Epoch 00800 | Train Loss 0.6057 | Val precision 39.1850% | Val recall 61.2745% | Val F1 47.8011%
Epoch 00900 | Train Loss 0.5949 | Val precision 38.3598% | Val recall 71.0784% | Val F1 49.8282%
Epoch 01000 | Train Loss 0.5925 | Val precision 38.1657% | Val recall 63.2353% | Val F1 47.6015%
Epoch 00100 | Train Loss 0.659

Epoch 00600 | Train Loss 0.6559 | Val precision 36.6197% | Val recall 63.7255% | Val F1 46.5116%
Epoch 00700 | Train Loss 0.6328 | Val precision 38.8715% | Val recall 60.7843% | Val F1 47.4187%
Epoch 00800 | Train Loss 0.6250 | Val precision 37.4670% | Val recall 69.6078% | Val F1 48.7136%
Epoch 00900 | Train Loss 0.6216 | Val precision 38.7812% | Val recall 68.6275% | Val F1 49.5575%
Epoch 01000 | Train Loss 0.6194 | Val precision 37.5959% | Val recall 72.0588% | Val F1 49.4118%
Epoch 00100 | Train Loss 0.6914 | Val precision 26.5280% | Val recall 100.0000% | Val F1 41.9322%
Epoch 00200 | Train Loss 0.6899 | Val precision 26.5280% | Val recall 100.0000% | Val F1 41.9322%
Epoch 00300 | Train Loss 0.6837 | Val precision 27.0667% | Val recall 99.5098% | Val F1 42.5577%
Epoch 00400 | Train Loss 0.6673 | Val precision 35.3887% | Val recall 64.7059% | Val F1 45.7539%
Epoch 00500 | Train Loss 0.6407 | Val precision 38.3686% | Val recall 62.2549% | Val F1 47.4766%
Epoch 00600 | Train Loss 0.6

Epoch 00100 | Train Loss 0.6910 | Val precision 26.5280% | Val recall 100.0000% | Val F1 41.9322%
Epoch 00200 | Train Loss 0.6878 | Val precision 26.5280% | Val recall 100.0000% | Val F1 41.9322%
Epoch 00300 | Train Loss 0.6759 | Val precision 30.5243% | Val recall 79.9020% | Val F1 44.1734%
Epoch 00400 | Train Loss 0.6537 | Val precision 31.8501% | Val recall 66.6667% | Val F1 43.1062%
Epoch 00500 | Train Loss 0.6387 | Val precision 34.6479% | Val recall 60.2941% | Val F1 44.0072%
Epoch 00600 | Train Loss 0.6321 | Val precision 35.0543% | Val recall 63.2353% | Val F1 45.1049%
Epoch 00700 | Train Loss 0.6280 | Val precision 35.1562% | Val recall 66.1765% | Val F1 45.9184%
Epoch 00800 | Train Loss 0.6246 | Val precision 35.4922% | Val recall 67.1569% | Val F1 46.4407%
Epoch 00900 | Train Loss 0.6218 | Val precision 35.7326% | Val recall 68.1373% | Val F1 46.8803%
Epoch 01000 | Train Loss 0.6198 | Val precision 36.6755% | Val recall 68.1373% | Val F1 47.6844%
Epoch 00100 | Train Loss 0.6

### Results of Graph CNN Optimisation : 

- APP Conv : with or without linear layer added generates a linearly decreasing loss and associated linearly decreasing accuracy with no good repartition of error between classes, difficulty to learn
- GCNN without linear layers: more unstable results, no learning 
- GCNN with linear layers: linear layer at the beginning help stabilize and learn, a second linear layer in front doesn't create significant impact |no linear layer at the end no learning, second linear layer at the end reduces learning/less balanced errors, performance around 75%
- Number of CNN layers: the addition of layers helps stabilize the learning accross epochs, when 4/5 layers less stables hence ~66% with more balanced, when 10/11 layers after 200 epochs very stales, errors not balanced at all
- Add dropout, increase instability, when in a max -> strong acc, bad repartition, when in a min, the opposite => removed 
- Addition of a Avgpooling layer: no significant improvement on accuracy or error repartition 
- Hidden layer size for GCNN: if increase layer size increase creates instability but at a certain extend balances the errors in classes, around 60
- Hidden layer size for final linear layer: 30, tradeoff with error-accuracy
- Cross entropy + Soft Max give very unstable results over trials, converge to all 0 or all 1
- NLL loss with log_sofmax gives very unstable results over epochs but reaches learning
- Adding weights to loss function does generate improvement 
- BCE Loss not appropriate
- Number of epochs, no need to go above 250, stabilisation around 200/250

=> if stable , accuracy around 75%, very unbalanced errors
=> if unstable (less layers, dropout ...), accuracy around 60%, more balanced errors 


### Useless

In [ ]:
# PCA without feature selection not significant impact
#pca = PCA(n_components=transformed_feat.shape[1])
#transformed_feat.iloc[train_mask] = pca.fit_transform(transformed_feat.iloc[train_mask].to_numpy())
#transformed_feat.iloc[val_mask] = pca.transform(transformed_feat.iloc[val_mask].to_numpy())